In [1]:
import pandas as pd

# アメダスデータを用いた　よく使うpandas機能の整理
- 本　Notebookは アメダスデータを用いて、Pandasでよく使ってる機能の整理を行う
- コードの綺麗さは追求しない
- pandasの機能にないのも一部含まれているが、私用のtips用に作成したものであるため、その点は目をつぶってほしい

まず札幌のアメダスデータで処理

In [2]:
# データの読み込み
df_s= pd.read_csv('./data/Sapporo_amedas.csv',skiprows=0,index_col=0,header=[1,2,4],encoding='cp932')##アメダスデータは特殊なencoding

In [3]:
df_s

札幌                                                \
年月日                   平均気温(℃)                   降水量の合計(mm)                    
           Unnamed: 1_level_2 品質情報 均質番号 Unnamed: 4_level_2 現象なし情報 品質情報 均質番号   
2020/1/1                 -5.8    8    1                2.5      0    8    1   
2020/1/2                 -4.0    8    1                0.0      0    8    1   
2020/1/3                 -2.5    8    1                0.5      0    8    1   
2020/1/4                 -5.0    8    1                1.5      0    8    1   
2020/1/5                 -4.7    8    1                2.0      0    8    1   
...                       ...  ...  ...                ...    ...  ...  ...   
2022/12/27                1.5    8    1                1.0      0    8    1   
2022/12/28                0.8    8    1                0.5      0    8    1   
2022/12/29               -2.3    8    1                4.0      0    8    1   
2022/12/30               -1.8    8    1                0.0      0    8    1   
2022/12/31                0.3    8    1                0.0      0    8    1   

                                         ...                      \
年月日             合計全天日射量(MJ/㎡)            ... 降雪量合計(cm)             
           Unnamed: 8_level_2 品質情報 均質番号  ...    現象なし情報 品質情報 均質番号   
2020/1/1                 3.44    8    1  ...         0    8    1   
2020/1/2                 4.63    8    1  ...         1    8    1   
2020/1/3                 5.36    8    1  ...         0    8    1   
2020/1/4                 7.32    8    1  ...         0    8    1   
2020/1/5                 4.51    8    1  ...         0    8    1   
...                       ...  ...  ...  ...       ...  ...  ...   
2022/12/27               4.62    8    1  ...         0    8    1   
2022/12/28               6.02    8    1  ...         0    8    1   
2022/12/29               4.14    8    1  ...         0    8    1   
2022/12/30               4.73    8    1  ...         1    8    1   
2022/12/31               4.79    8    1  ...         1    8    1   

                                                                               
年月日                平均現地気圧(hPa)                      日照時間(時間)                   
           Unnamed: 25_level_2 品質情報 均質番号 Unnamed: 28_level_2 現象なし情報 品質情報 均質番号  
2020/1/1                1009.3    8    1                 0.2      0    8    1  
2020/1/2                1011.1    8    1                 1.2      0    8    1  
2020/1/3                1008.6    8    1                 1.6      0    8    1  
2020/1/4                1010.4    8    1                 5.2      0    8    1  
2020/1/5                1015.0    8    1                 2.2      0    8    1  
...                        ...  ...  ...                 ...    ...  ...  ...  
2022/12/27              1016.5    8    1                 1.8      0    8    1  
2022/12/28              1013.8    8    1                 3.4      0    8    1  
2022/12/29              1009.7    8    1                 2.3      0    8    1  
2022/12/30              1015.2    8    1                 3.1      0    8    1  
2022/12/31              1014.9    8    1                 1.2      0    8    1  

[1096 rows x 31 columns]

## multi columnsの中に入っている　Unnamed:というものを除去した上で データクレンジングを行う処理

In [4]:
# Unnamedが邪魔なので消す関数の作成
def multi_columns(data_frame):
    df = data_frame.copy()
    if type(df.columns) is not pd.MultiIndex:
        return df

    # 列にUnnamedという文字の入った内容を削除しインデックスを振り直す
    df = df.rename(columns=lambda x: x if not 'Unnamed' in str(x) else '')
    df = df.reset_index()

    cols = df.columns
    copy_col = list(cols)

    # column.namesを新しいcolumnの一番上(インデックスセルの一番左)に持ってくる
    name_col = tuple([(name if name is not None else '') for name in cols.names])
    copy_col[0] = name_col

    # 無効セルは上に詰める
    for i, col in enumerate(copy_col):
        pack = [content for content in col if content != ""]
        copy_col[i] = tuple(pack + ([""] * (len(col)  - len(pack))))

    df.columns = pd.MultiIndex.from_tuples(copy_col)
    cols.names = tuple([None for x in cols.names])
    return df

In [5]:
df_s=multi_columns(df_s)

In [6]:
#カラム名の確認
df_s.columns

MultiIndex([('年月日',              '',       ''),
            ( '札幌',       '平均気温(℃)',       ''),
            ( '札幌',       '平均気温(℃)',   '品質情報'),
            ( '札幌',       '平均気温(℃)',   '均質番号'),
            ( '札幌',    '降水量の合計(mm)',       ''),
            ( '札幌',    '降水量の合計(mm)', '現象なし情報'),
            ( '札幌',    '降水量の合計(mm)',   '品質情報'),
            ( '札幌',    '降水量の合計(mm)',   '均質番号'),
            ( '札幌', '合計全天日射量(MJ/㎡)',       ''),
            ( '札幌', '合計全天日射量(MJ/㎡)',   '品質情報'),
            ( '札幌', '合計全天日射量(MJ/㎡)',   '均質番号'),
            ( '札幌',      '最深積雪(cm)',       ''),
            ( '札幌',      '最深積雪(cm)', '現象なし情報'),
            ( '札幌',      '最深積雪(cm)',   '品質情報'),
            ( '札幌',      '最深積雪(cm)',   '均質番号'),
            ( '札幌',     '平均風速(m/s)',       ''),
            ( '札幌',     '平均風速(m/s)',   '品質情報'),
            ( '札幌',     '平均風速(m/s)',   '均質番号'),
            ( '札幌',    '平均雲量(10分比)',       ''),
            ( '札幌',    '平均雲量(10分比)',   '品質情報'),
            ( '札幌',    '平均雲量(10分比)',   '

In [7]:
#日付をdatetime型に揃えて、日付を　index化ける
df_s.loc[:, (('年月日',              '',       ''))] = pd.to_datetime(df_s[(('年月日',              '',       ''))])
df_s.set_index(('年月日',              '',       ''),inplace=True)

## マルチカラムインデックスから、不要な列を削除し、列名に気象台を設ける

In [8]:
#　均質番号などの不要な列は削除する
df_s= df_s.loc[:,(slice(None),slice(None),'')].copy()
df_s.head()

札幌                                                         \
           平均気温(℃) 降水量の合計(mm) 合計全天日射量(MJ/㎡) 最深積雪(cm) 平均風速(m/s) 平均雲量(10分比)   
                                                                            
(年月日, , )                                                                   
2020-01-01    -5.8        2.5          3.44        6       1.8        9.8   
2020-01-02    -4.0        0.0          4.63        4       2.3        7.5   
2020-01-03    -2.5        0.5          5.36        4       3.4        9.8   
2020-01-04    -5.0        1.5          7.32        6       3.4        6.8   
2020-01-05    -4.7        2.0          4.51        8       3.6        5.8   

                                           
           降雪量合計(cm) 平均現地気圧(hPa) 日照時間(時間)  
                                           
(年月日, , )                                  
2020-01-01         5      1009.3      0.2  
2020-01-02         0      1011.1      1.2  
2020-01-03         1      1008.6      1.6  
2020-01-04         2      1010.4      5.2  
2020-01-05         4      1015.0      2.2

In [9]:
#気象台名を付け加える（他のデータとの連結も兼ねて）
df1= df_s.stack(level=0)##level0は0行目
print(df1.columns)
df1.head()

MultiIndex([('合計全天日射量(MJ/㎡)', ''),
            (      '平均気温(℃)', ''),
            (  '平均現地気圧(hPa)', ''),
            (   '平均雲量(10分比)', ''),
            (    '平均風速(m/s)', ''),
            (     '日照時間(時間)', ''),
            (     '最深積雪(cm)', ''),
            (   '降水量の合計(mm)', ''),
            (    '降雪量合計(cm)', '')],
           )


,,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
,,,,,,,,,,
"(年月日, , )",,,,,,,,,,
2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5
2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0
2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1
2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2
2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4


In [10]:
#一番上の要素だけ取得
df1.columns=[element[0] for element in df1.columns]
df1.head()

,,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
"(年月日, , )",,,,,,,,,,
2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5
2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0
2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1
2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2
2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4


In [11]:
#　気象台の列名を付与
df_1=df1.reset_index(inplace=False)
df_1.rename(columns={('年月日', '', ''):'日付','level_1':'気象台名'},inplace=True)
df_1.head()

,日付,気象台名,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
0,2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5
1,2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0
2,2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1
3,2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2
4,2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4


## 別の気象台とのデータ連結
- 名古屋の気象台の値を持ってきて、データ連結を行う
-　練習用のため,pandasのmerge,join,concatの３パターンでデータを連結させる

In [12]:
# データの読み込み
df_n= pd.read_csv('./data/Nagoya_amedas.csv',skiprows=0,index_col=0,header=[1,2,4],encoding='cp932')##アメダスデータは特殊なencoding
df_n

名古屋                                                \
年月日                   平均気温(℃)                   降水量の合計(mm)                    
           Unnamed: 1_level_2 品質情報 均質番号 Unnamed: 4_level_2 現象なし情報 品質情報 均質番号   
2020/1/1                  5.3    8    1                0.0      0    8    1   
2020/1/2                  7.6    8    1                0.0      0    8    1   
2020/1/3                  8.1    8    1                0.0      0    8    1   
2020/1/4                  7.7    8    1                0.0      0    8    1   
2020/1/5                  5.8    8    1                0.0      1    8    1   
...                       ...  ...  ...                ...    ...  ...  ...   
2022/12/27                5.6    8    1                0.0      1    8    1   
2022/12/28                5.3    8    1                0.0      1    8    1   
2022/12/29                5.9    8    1                0.0      0    8    1   
2022/12/30                5.6    8    1                0.0      0    8    1   
2022/12/31                5.8    8    1                0.0      1    8    1   

                                         ...                      \
年月日             合計全天日射量(MJ/㎡)            ... 降雪量合計(cm)             
           Unnamed: 8_level_2 品質情報 均質番号  ...    現象なし情報 品質情報 均質番号   
2020/1/1                 8.50    8    1  ...         1    8    1   
2020/1/2                 9.33    8    1  ...         1    8    1   
2020/1/3                 6.34    8    1  ...         1    8    1   
2020/1/4                 8.11    8    1  ...         1    8    1   
2020/1/5                12.99    8    1  ...         1    8    1   
...                       ...  ...  ...  ...       ...  ...  ...   
2022/12/27              11.70    8    1  ...         1    8    1   
2022/12/28              11.72    8    1  ...         1    8    1   
2022/12/29               6.54    8    1  ...         1    8    1   
2022/12/30               6.11    8    1  ...         1    8    1   
2022/12/31               7.29    8    1  ...         1    8    1   

                                                                               
年月日                平均現地気圧(hPa)                      日照時間(時間)                   
           Unnamed: 25_level_2 品質情報 均質番号 Unnamed: 28_level_2 現象なし情報 品質情報 均質番号  
2020/1/1                1017.9    8    1                 5.1      0    8    1  
2020/1/2                1016.7    8    1                 4.9      0    8    1  
2020/1/3                1012.7    8    1                 3.0      0    8    1  
2020/1/4                1009.8    8    1                 3.0      0    8    1  
2020/1/5                1017.5    8    1                 9.3      0    8    1  
...                        ...  ...  ...                 ...    ...  ...  ...  
2022/12/27              1016.5    8    1                 8.0      0    8    1  
2022/12/28              1017.4    8    1                 9.2      0    8    1  
2022/12/29              1017.2    8    1                 3.1      0    8    1  
2022/12/30              1019.0    8    1                 2.5      0    8    1  
2022/12/31              1017.8    8    1                 2.9      0    8    1  

[1096 rows x 31 columns]

札幌と同様に、unnnamedの削除,気象台の列追加の前処理

In [13]:
#マルチカラム削除
df_n=multi_columns(df_n)
#日付をdatetime型に揃えて、日付を　index化ける
df_n.loc[:, (('年月日',              '',       ''))] = pd.to_datetime(df_n[(('年月日',              '',       ''))])
df_n.set_index(('年月日',              '',       ''),inplace=True)
#　均質番号などの不要な列は削除する
df_n= df_n.loc[:,(slice(None),slice(None),'')].copy()
#気象台名を付け加える（他のデータとの連結も兼ねて）
df2= df_n.stack(level=0)##level0は0行目
#一番上の要素だけ取得
df2.columns=[element[0] for element in df2.columns]
#　気象台の列名を付与
df_2=df2.reset_index(inplace=False)
df_2.rename(columns={(('年月日', '', '')):'日付','level_1':'気象台名'},inplace=True)
df_2.head()

,日付,気象台名,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
0,2020-01-01,名古屋,8.50,5.3,1017.9,5.3,3.3,5.1,0,0.0,0
1,2020-01-02,名古屋,9.33,7.6,1016.7,8.8,2.6,4.9,0,0.0,0
2,2020-01-03,名古屋,6.34,8.1,1012.7,7.0,2.6,3.0,0,0.0,0
3,2020-01-04,名古屋,8.11,7.7,1009.8,10.0,3.2,3.0,0,0.0,0
4,2020-01-05,名古屋,12.99,5.8,1017.5,0.8,5.2,9.3,0,0.0,0


### ① pd mergeの場合
- デメリット：x_y_のリネームが必要
- メリット:onを定めると

札幌が　df_1,名古屋がdf_2で解析

In [14]:
df_merge=pd.merge(df_1,df_2,on='日付')
print(df_merge.columns)
df_merge.head()

Index(['日付', '気象台名_x', '合計全天日射量(MJ/㎡)_x', '平均気温(℃)_x', '平均現地気圧(hPa)_x',
       '平均雲量(10分比)_x', '平均風速(m/s)_x', '日照時間(時間)_x', '最深積雪(cm)_x',
       '降水量の合計(mm)_x', '降雪量合計(cm)_x', '気象台名_y', '合計全天日射量(MJ/㎡)_y', '平均気温(℃)_y',
       '平均現地気圧(hPa)_y', '平均雲量(10分比)_y', '平均風速(m/s)_y', '日照時間(時間)_y',
       '最深積雪(cm)_y', '降水量の合計(mm)_y', '降雪量合計(cm)_y'],
      dtype='object')


,日付,気象台名_x,合計全天日射量(MJ/㎡)_x,平均気温(℃)_x,平均現地気圧(hPa)_x,平均雲量(10分比)_x,平均風速(m/s)_x,日照時間(時間)_x,最深積雪(cm)_x,降水量の合計(mm)_x,...,気象台名_y,合計全天日射量(MJ/㎡)_y,平均気温(℃)_y,平均現地気圧(hPa)_y,平均雲量(10分比)_y,平均風速(m/s)_y,日照時間(時間)_y,最深積雪(cm)_y,降水量の合計(mm)_y,降雪量合計(cm)_y
0,2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,...,名古屋,8.50,5.3,1017.9,5.3,3.3,5.1,0,0.0,0
1,2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,...,名古屋,9.33,7.6,1016.7,8.8,2.6,4.9,0,0.0,0
2,2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,...,名古屋,6.34,8.1,1012.7,7.0,2.6,3.0,0,0.0,0
3,2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,...,名古屋,8.11,7.7,1009.8,10.0,3.2,3.0,0,0.0,0
4,2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,...,名古屋,12.99,5.8,1017.5,0.8,5.2,9.3,0,0.0,0


### ②pd. concatの場合
- デメリット：列をそのまま結合しているので、1つでも一致しないとNaNになる可能性がある
- メリット: 気軽,列のリネームが不要,列の形態が一致しているとき有用
→今回は使わないのでコメントアウト

In [38]:
#df_concat=pd.concat([df_1,df_2],axis=1)
#print(df_concat.columns)
#df_concat.head()

### ③join関数の場合
- デメリット:列名がoverlapしてしまうと使えない（なのでエラー文が出てしまうのでコメントアウト）
- エラー文:ValueError: columns overlap but no suffix specified: Index(['日付', '気象台名', '合計全天日射量(MJ/㎡)', '平均気温(℃)', '平均現地気圧(hPa)', '平均雲量(10分比)',
       '平均風速(m/s)', '日照時間(時間)', '最深積雪(cm)', '降水量の合計(mm)', '降雪量合計(cm)'],
      dtype='object')

In [16]:
#df_join=df_1.join(df_2)

### merge関数の場合での列名の部分的リネーム
- 今回の場合最も実用的なmergeに対して、renameをする
-　_xは札幌,_yは名古屋に変更　列名の一般的なrenameは使えないので　関数化する

In [17]:
#コラムのインデントのスペース排除
def process_str_fmt(string:str) -> str:
    """_xは札幌,_yは名古屋"""
    string = string.replace('_x', '_札幌')
    string = string.replace('_y', '_名古屋')
    return string

In [18]:
df_merge.columns = [process_str_fmt(col) for col in df_merge.columns]
df_merge.head()

,日付,気象台名_札幌,合計全天日射量(MJ/㎡)_札幌,平均気温(℃)_札幌,平均現地気圧(hPa)_札幌,平均雲量(10分比)_札幌,平均風速(m/s)_札幌,日照時間(時間)_札幌,最深積雪(cm)_札幌,降水量の合計(mm)_札幌,...,気象台名_名古屋,合計全天日射量(MJ/㎡)_名古屋,平均気温(℃)_名古屋,平均現地気圧(hPa)_名古屋,平均雲量(10分比)_名古屋,平均風速(m/s)_名古屋,日照時間(時間)_名古屋,最深積雪(cm)_名古屋,降水量の合計(mm)_名古屋,降雪量合計(cm)_名古屋
0,2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,...,名古屋,8.50,5.3,1017.9,5.3,3.3,5.1,0,0.0,0
1,2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,...,名古屋,9.33,7.6,1016.7,8.8,2.6,4.9,0,0.0,0
2,2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,...,名古屋,6.34,8.1,1012.7,7.0,2.6,3.0,0,0.0,0
3,2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,...,名古屋,8.11,7.7,1009.8,10.0,3.2,3.0,0,0.0,0
4,2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,...,名古屋,12.99,5.8,1017.5,0.8,5.2,9.3,0,0.0,0


In [19]:
## 気象台名は列に入っているので削除します
df_merge.drop(columns=['気象台名_札幌','気象台名_名古屋'],inplace=True)

In [20]:
df_merge

,日付,合計全天日射量(MJ/㎡)_札幌,平均気温(℃)_札幌,平均現地気圧(hPa)_札幌,平均雲量(10分比)_札幌,平均風速(m/s)_札幌,日照時間(時間)_札幌,最深積雪(cm)_札幌,降水量の合計(mm)_札幌,降雪量合計(cm)_札幌,合計全天日射量(MJ/㎡)_名古屋,平均気温(℃)_名古屋,平均現地気圧(hPa)_名古屋,平均雲量(10分比)_名古屋,平均風速(m/s)_名古屋,日照時間(時間)_名古屋,最深積雪(cm)_名古屋,降水量の合計(mm)_名古屋,降雪量合計(cm)_名古屋
0,2020-01-01,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5,8.50,5.3,1017.9,5.3,3.3,5.1,0,0.0,0
1,2020-01-02,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0,9.33,7.6,1016.7,8.8,2.6,4.9,0,0.0,0
2,2020-01-03,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1,6.34,8.1,1012.7,7.0,2.6,3.0,0,0.0,0
3,2020-01-04,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2,8.11,7.7,1009.8,10.0,3.2,3.0,0,0.0,0
4,2020-01-05,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4,12.99,5.8,1017.5,0.8,5.2,9.3,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2022-12-27,4.62,1.5,1016.5,9.5,1.4,1.8,23,1.0,2,11.70,5.6,1016.5,3.8,3.4,8.0,0,0.0,0
1092,2022-12-28,6.02,0.8,1013.8,7.5,2.0,3.4,22,0.5,1,11.72,5.3,1017.4,0.8,1.8,9.2,0,0.0,0
1093,2022-12-29,4.14,-2.3,1009.7,8.5,3.4,2.3,29,4.0,7,6.54,5.9,1017.2,4.5,3.5,3.1,0,0.0,0
1094,2022-12-30,4.73,-1.8,1015.2,7.5,2.9,3.1,29,0.0,0,6.11,5.6,1019.0,8.8,3.5,2.5,0,0.0,0


## 　lagをとって、前の値自体、差分、標準偏差をとる
- 5つ前までの値（5日前）の、平均気温札幌で実験

In [21]:
#前の値
before_lag=[1,2,3,4,5]
#予測発表時点から1時点前のlaptime 
for lag in before_lag:
    df_merge = pd.concat(
        [df_merge,df_merge['平均気温(℃)_札幌'].shift(lag).rename(str(lag)+'日前_平均気温(℃)_札幌')],
        axis=1
    ) 
df_merge.head()

,日付,合計全天日射量(MJ/㎡)_札幌,平均気温(℃)_札幌,平均現地気圧(hPa)_札幌,平均雲量(10分比)_札幌,平均風速(m/s)_札幌,日照時間(時間)_札幌,最深積雪(cm)_札幌,降水量の合計(mm)_札幌,降雪量合計(cm)_札幌,...,平均風速(m/s)_名古屋,日照時間(時間)_名古屋,最深積雪(cm)_名古屋,降水量の合計(mm)_名古屋,降雪量合計(cm)_名古屋,1日前_平均気温(℃)_札幌,2日前_平均気温(℃)_札幌,3日前_平均気温(℃)_札幌,4日前_平均気温(℃)_札幌,5日前_平均気温(℃)_札幌
0,2020-01-01,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5,...,3.3,5.1,0,0.0,0,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0,...,2.6,4.9,0,0.0,0,-5.8,NaN,NaN,NaN,NaN
2,2020-01-03,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1,...,2.6,3.0,0,0.0,0,-4.0,-5.8,NaN,NaN,NaN
3,2020-01-04,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2,...,3.2,3.0,0,0.0,0,-2.5,-4.0,-5.8,NaN,NaN
4,2020-01-05,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4,...,5.2,9.3,0,0.0,0,-5.0,-2.5,-4.0,-5.8,NaN


In [22]:
#差分
for lag in before_lag:
    df_merge = pd.concat(
        [df_merge,df_merge['平均気温(℃)_札幌'].diff(lag).rename(str(lag)+'日前_平均気温(℃)の差分_札幌')],
        axis=1
    ) 
df_merge.head()

,日付,合計全天日射量(MJ/㎡)_札幌,平均気温(℃)_札幌,平均現地気圧(hPa)_札幌,平均雲量(10分比)_札幌,平均風速(m/s)_札幌,日照時間(時間)_札幌,最深積雪(cm)_札幌,降水量の合計(mm)_札幌,降雪量合計(cm)_札幌,...,1日前_平均気温(℃)_札幌,2日前_平均気温(℃)_札幌,3日前_平均気温(℃)_札幌,4日前_平均気温(℃)_札幌,5日前_平均気温(℃)_札幌,1日前_平均気温(℃)の差分_札幌,2日前_平均気温(℃)の差分_札幌,3日前_平均気温(℃)の差分_札幌,4日前_平均気温(℃)の差分_札幌,5日前_平均気温(℃)の差分_札幌
0,2020-01-01,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0,...,-5.8,NaN,NaN,NaN,NaN,1.8,NaN,NaN,NaN,NaN
2,2020-01-03,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1,...,-4.0,-5.8,NaN,NaN,NaN,1.5,3.3,NaN,NaN,NaN
3,2020-01-04,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2,...,-2.5,-4.0,-5.8,NaN,NaN,-2.5,-1.0,0.8,NaN,NaN
4,2020-01-05,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4,...,-5.0,-2.5,-4.0,-5.8,NaN,0.3,-2.2,-0.7,1.1,NaN


In [23]:
#標準偏差
for lag in before_lag:
    if lag>=2:
        df_merge = pd.concat(
            [df_merge,df_merge['平均気温(℃)_札幌'].rolling(lag).std().rename(str(lag)+'日前_平均気温(℃)の標準偏差_札幌')],
            axis=1) 
df_merge.head()

,日付,合計全天日射量(MJ/㎡)_札幌,平均気温(℃)_札幌,平均現地気圧(hPa)_札幌,平均雲量(10分比)_札幌,平均風速(m/s)_札幌,日照時間(時間)_札幌,最深積雪(cm)_札幌,降水量の合計(mm)_札幌,降雪量合計(cm)_札幌,...,5日前_平均気温(℃)_札幌,1日前_平均気温(℃)の差分_札幌,2日前_平均気温(℃)の差分_札幌,3日前_平均気温(℃)の差分_札幌,4日前_平均気温(℃)の差分_札幌,5日前_平均気温(℃)の差分_札幌,2日前_平均気温(℃)の標準偏差_札幌,3日前_平均気温(℃)の標準偏差_札幌,4日前_平均気温(℃)の標準偏差_札幌,5日前_平均気温(℃)の標準偏差_札幌
0,2020-01-01,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0,...,NaN,1.8,NaN,NaN,NaN,NaN,1.272792,NaN,NaN,NaN
2,2020-01-03,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1,...,NaN,1.5,3.3,NaN,NaN,NaN,1.060660,1.652271,NaN,NaN
3,2020-01-04,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2,...,NaN,-2.5,-1.0,0.8,NaN,NaN,1.767767,1.258306,1.422146,NaN
4,2020-01-05,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4,...,NaN,0.3,-2.2,-0.7,1.1,NaN,0.212132,1.365040,1.115049,1.24298


In [24]:
print(df_merge.columns)

Index(['日付', '合計全天日射量(MJ/㎡)_札幌', '平均気温(℃)_札幌', '平均現地気圧(hPa)_札幌',
       '平均雲量(10分比)_札幌', '平均風速(m/s)_札幌', '日照時間(時間)_札幌', '最深積雪(cm)_札幌',
       '降水量の合計(mm)_札幌', '降雪量合計(cm)_札幌', '合計全天日射量(MJ/㎡)_名古屋', '平均気温(℃)_名古屋',
       '平均現地気圧(hPa)_名古屋', '平均雲量(10分比)_名古屋', '平均風速(m/s)_名古屋', '日照時間(時間)_名古屋',
       '最深積雪(cm)_名古屋', '降水量の合計(mm)_名古屋', '降雪量合計(cm)_名古屋', '1日前_平均気温(℃)_札幌',
       '2日前_平均気温(℃)_札幌', '3日前_平均気温(℃)_札幌', '4日前_平均気温(℃)_札幌', '5日前_平均気温(℃)_札幌',
       '1日前_平均気温(℃)の差分_札幌', '2日前_平均気温(℃)の差分_札幌', '3日前_平均気温(℃)の差分_札幌',
       '4日前_平均気温(℃)の差分_札幌', '5日前_平均気温(℃)の差分_札幌', '2日前_平均気温(℃)の標準偏差_札幌',
       '3日前_平均気温(℃)の標準偏差_札幌', '4日前_平均気温(℃)の標準偏差_札幌', '5日前_平均気温(℃)の標準偏差_札幌'],
      dtype='object')


## 気象台名ごとにgroupbyをし、平均等々をとってみる

In [26]:
df_1 .head()

,日付,気象台名,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
0,2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5
1,2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0
2,2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1
3,2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2
4,2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4


In [27]:
df_2.head()

,日付,気象台名,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
0,2020-01-01,名古屋,8.50,5.3,1017.9,5.3,3.3,5.1,0,0.0,0
1,2020-01-02,名古屋,9.33,7.6,1016.7,8.8,2.6,4.9,0,0.0,0
2,2020-01-03,名古屋,6.34,8.1,1012.7,7.0,2.6,3.0,0,0.0,0
3,2020-01-04,名古屋,8.11,7.7,1009.8,10.0,3.2,3.0,0,0.0,0
4,2020-01-05,名古屋,12.99,5.8,1017.5,0.8,5.2,9.3,0,0.0,0


Groupby用に　縦連結させます

In [30]:
df_concat2=pd.concat([df_1,df_2],axis=0)
df_concat2

,日付,気象台名,合計全天日射量(MJ/㎡),平均気温(℃),平均現地気圧(hPa),平均雲量(10分比),平均風速(m/s),日照時間(時間),最深積雪(cm),降水量の合計(mm),降雪量合計(cm)
0,2020-01-01,札幌,3.44,-5.8,1009.3,9.8,1.8,0.2,6,2.5,5
1,2020-01-02,札幌,4.63,-4.0,1011.1,7.5,2.3,1.2,4,0.0,0
2,2020-01-03,札幌,5.36,-2.5,1008.6,9.8,3.4,1.6,4,0.5,1
3,2020-01-04,札幌,7.32,-5.0,1010.4,6.8,3.4,5.2,6,1.5,2
4,2020-01-05,札幌,4.51,-4.7,1015.0,5.8,3.6,2.2,8,2.0,4
...,...,...,...,...,...,...,...,...,...,...,...
1091,2022-12-27,名古屋,11.70,5.6,1016.5,3.8,3.4,8.0,0,0.0,0
1092,2022-12-28,名古屋,11.72,5.3,1017.4,0.8,1.8,9.2,0,0.0,0
1093,2022-12-29,名古屋,6.54,5.9,1017.2,4.5,3.5,3.1,0,0.0,0
1094,2022-12-30,名古屋,6.11,5.6,1019.0,8.8,3.5,2.5,0,0.0,0


In [37]:
df_group=df_concat2.groupby(['日付','気象台名']).mean()
df_group.head()

合計全天日射量(MJ/㎡)  平均気温(℃)  平均現地気圧(hPa)  平均雲量(10分比)  平均風速(m/s)  \
日付         気象台名                                                               
2020-01-01 名古屋            8.50      5.3       1017.9         5.3        3.3   
           札幌             3.44     -5.8       1009.3         9.8        1.8   
2020-01-02 名古屋            9.33      7.6       1016.7         8.8        2.6   
           札幌             4.63     -4.0       1011.1         7.5        2.3   
2020-01-03 名古屋            6.34      8.1       1012.7         7.0        2.6   

                 日照時間(時間)  最深積雪(cm)  降水量の合計(mm)  降雪量合計(cm)  
日付         気象台名                                             
2020-01-01 名古屋        5.1       0.0         0.0        0.0  
           札幌         0.2       6.0         2.5        5.0  
2020-01-02 名古屋        4.9       0.0         0.0        0.0  
           札幌         1.2       4.0         0.0        0.0  
2020-01-03 名古屋        3.0       0.0         0.0        0.0